In [ ]:
import matplotlib.pyplot as plt   #implementing the plotting library
import numpy as np  #including the num array library
import pandas as pd #including pandas library (necessary for some functionalities)
from sklearn.linear_model import Ridge #used to estimate to predict the weather
from sklearn.preprocessing import LabelEncoder #used to encode the data
url = "https://drive.usercontent.google.com/u/0/uc?id=1faupcREQNNpea0SrMohTmyABQKdUJJUn&export=download" #url of the data
pred = pd.read_csv(url) #reading the data from the url

In [ ]:
pred.dropna(inplace=True) #removing the null values
pred.drop_duplicates(inplace=True) #removing the duplicate values
pred.set_index("Formatted Date",inplace=True) #setting the index to the date
pred.ffill(inplace=True) #forward filling the missing values
pred.index = pd.to_datetime(pred.index) #converting the index to the datetime format
pred.sort_index(inplace=True) #sorting the index

In [ ]:
le = LabelEncoder() #initializing the label encoder
pred["next_temp"] = pred["Temperature (C)"].shift(-1) #creating a new column with the next temperature
pred["next_apparent_temp"] = pred["Apparent Temperature (C)"].shift(-1) #creating a new column with the next apparent temperature
pred["next_humidity"] = pred["Humidity"].shift(-1) #creating a new column with the next humidity
pred["next_wind_speed"] = pred["Wind Speed (km/h)"].shift(-1) #creating a new column with the next wind speed
pred["next_wind_bearing"] = pred["Wind Bearing (degrees)"].shift(-1) #creating a new column with the next wind bearing
pred["next_visibility"] = pred["Visibility (km)"].shift(-1) #creating a new column with the next visibility
pred["next_pressure"] = pred["Pressure (millibars)"].shift(-1) #creating a new column with the next pressure
pred['Formatted Date'] = le.fit_transform(pred['Formatted Date']) #encoding the date
pred['Summary'] = le.fit_transform(pred['Summary']) #encoding the summary
pred['Precip Type'] = le.fit_transform(pred['Precip Type']) #encoding the precip type
pred['Daily Summary'] = le.fit_transform(pred['Daily Summary']) #encoding the daily summary
pred['Loud Cover'] = le.fit_transform(pred['Loud Cover']) #encoding the loud cover
pred.dropna(inplace=True) #removing the null values

In [ ]:
setup = Ridge(alpha=0.1) #initializing the ridge regression model
estimators = pred.columns[~pred.columns.isin(["next_pressure","next_visibility","next_wind_bearing","next_wind_speed","next_humidity","next_apparent_temp","next_temp","Summary","Precip Type","Daily Summary","Loud Cover"])] #selecting the estimators


In [ ]:
def testing(pred,model,estimators,start = 3650, step = 120):
  all_pred = []

  for i in range(start,pred.shape[0],step): #iterating through the data
    training_model = pred.iloc[:i,:]  #training the model
    testing_model = pred.iloc[i:(i+step),:] #testing of the model
    model.fit(training_model[estimators],training_model["next_temp"]) #fitting the model
    pred_weather = model.predict(testing_model[estimators]) #predicting the weather
    pred_weather = pd.Series(pred_weather,index = testing_model.index) #converting the predicted weather to a series
    combined_weather = pd.concat([testing_model["next_temp"],pred_weather],axis = 1) #combining the actual and predicted weather
    combined_weather.columns = ["temp","pred_temp"] #renaming the columns
    combined_weather["diff_temp"] = (combined_weather["pred_temp"] - combined_weather["temp"]).abs() #calculating the difference between the actual and predicted weather
    #with every numerical feature (apparent temperature, pressure, ...etc) we will predict in the same way.
    model.fit(training_model[estimators],training_model["next_apparent_temp"])
    pred_weather = model.predict(testing_model[estimators])
    pred_weather = pd.Series(pred_weather,index = testing_model.index)
    combined_weather["real_apparent_temp"] = testing_model["Apparent Temperature (C)"]
    combined_weather["pred_apparent_temp"] = pred_weather
    combined_weather["diff_apparent_temp"] = (combined_weather["pred_apparent_temp"] - combined_weather["real_apparent_temp"]).abs()
    model.fit(training_model[estimators],training_model["next_humidity"])
    pred_weather = model.predict(testing_model[estimators])
    pred_weather = pd.Series(pred_weather,index = testing_model.index)
    combined_weather["real_humidity"] = testing_model["Humidity"]
    combined_weather["pred_humidity"] = pred_weather
    combined_weather["diff_humidity"] = (combined_weather["pred_humidity"] - combined_weather["real_humidity"]).abs()
    model.fit(training_model[estimators],training_model["next_wind_speed"])
    pred_weather = model.predict(testing_model[estimators])
    pred_weather = pd.Series(pred_weather,index = testing_model.index)
    combined_weather["real_wind_speed"] = testing_model["Wind Speed (km/h)"]
    combined_weather["pred_wind_speed"] = pred_weather
    combined_weather["diff_wind_speed"] = (combined_weather["pred_wind_speed"] - combined_weather["real_wind_speed"]).abs()
    model.fit(training_model[estimators],training_model["next_wind_bearing"])
    pred_weather = model.predict(testing_model[estimators])
    pred_weather = pd.Series(pred_weather,index = testing_model.index)
    combined_weather["real_wind_bearing"] = testing_model["Wind Bearing (degrees)"]
    combined_weather["pred_wind_bearing"] = pred_weather
    combined_weather["diff_wind_bearing"] = (combined_weather["pred_wind_bearing"] - combined_weather["real_wind_bearing"]).abs()
    model.fit(training_model[estimators],training_model["next_visibility"])
    pred_weather = model.predict(testing_model[estimators])
    pred_weather = pd.Series(pred_weather,index = testing_model.index)
    combined_weather["real_visibility"] = testing_model["Visibility (km)"]
    combined_weather["pred_visibility"] = pred_weather
    combined_weather["diff_visibility"] = (combined_weather["pred_visibility"] - combined_weather["real_visibility"]).abs()
    model.fit(training_model[estimators],training_model["next_pressure"])
    pred_weather = model.predict(testing_model[estimators])
    pred_weather = pd.Series(pred_weather,index = testing_model.index)
    combined_weather["real_pressure"] = testing_model["Pressure (millibars)"]
    combined_weather["pred_pressure"] = pred_weather
    combined_weather["diff_pressure"] = (combined_weather["pred_pressure"] - combined_weather["real_pressure"]).abs()
    all_pred.append(combined_weather)
  return pd.concat(all_pred)

In [ ]:
predictions = testing(pred,setup,estimators)
predictions

temp  pred_temp  diff_temp  \
1970-01-01 00:00:00.000003650  15.094444  17.558114   2.463669   
1970-01-01 00:00:00.000003651  14.005556  14.950248   0.944693   
1970-01-01 00:00:00.000003652  14.200000  13.975045   0.224955   
1970-01-01 00:00:00.000003653  16.094444  14.316194   1.778251   
1970-01-01 00:00:00.000003654  21.905556  16.152604   5.752952   
...                                  ...        ...        ...   
1970-01-01 00:00:00.000096447  26.016667  28.396681   2.380014   
1970-01-01 00:00:00.000096448  24.583333  25.675807   1.092474   
1970-01-01 00:00:00.000096449  22.038889  24.298941   2.260052   
1970-01-01 00:00:00.000096450  21.522222  21.792763   0.270541   
1970-01-01 00:00:00.000096451  20.438889  21.321472   0.882584   

                               real_apparent_temp  pred_apparent_temp  \
1970-01-01 00:00:00.000003650           17.611111           17.724304   
1970-01-01 00:00:00.000003651           15.094444           14.866866   
1970-01-01 00:00:00.000003652           14.005556           13.988637   
1970-01-01 00:00:00.000003653           14.200000           14.306789   
1970-01-01 00:00:00.000003654           16.094444           16.179954   
...                                           ...                 ...   
1970-01-01 00:00:00.000096447           27.850000           27.698166   
1970-01-01 00:00:00.000096448           26.016667           25.724065   
1970-01-01 00:00:00.000096449           24.583333           24.327593   
1970-01-01 00:00:00.000096450           22.038889           21.784810   
1970-01-01 00:00:00.000096451           21.522222           21.305987   

                               diff_apparent_temp  real_humidity  \
1970-01-01 00:00:00.000003650            0.113193           0.93   
1970-01-01 00:00:00.000003651            0.227579           0.96   
1970-01-01 00:00:00.000003652            0.016918           0.96   
1970-01-01 00:00:00.000003653            0.106789           0.95   
1970-01-01 00:00:00.000003654            0.085510           1.00   
...                                           ...            ...   
1970-01-01 00:00:00.000096447            0.151834           0.32   
1970-01-01 00:00:00.000096448            0.292601           0.43   
1970-01-01 00:00:00.000096449            0.255740           0.48   
1970-01-01 00:00:00.000096450            0.254079           0.56   
1970-01-01 00:00:00.000096451            0.216235           0.60   

                               pred_humidity  diff_humidity  real_wind_speed  \
1970-01-01 00:00:00.000003650       0.923600       0.006400           2.9946   
1970-01-01 00:00:00.000003651       0.956613       0.003387           3.2200   
1970-01-01 00:00:00.000003652       0.953670       0.006330           3.2200   
1970-01-01 00:00:00.000003653       0.936553       0.013447           3.1395   
1970-01-01 00:00:00.000003654       0.985323       0.014677           0.1288   
...                                      ...            ...              ...   
1970-01-01 00:00:00.000096447       0.342960       0.022960          12.2038   
1970-01-01 00:00:00.000096448       0.447801       0.017801          10.9963   
1970-01-01 00:00:00.000096449       0.494971       0.014971          10.0947   
1970-01-01 00:00:00.000096450       0.572379       0.012379           8.9838   
1970-01-01 00:00:00.000096451       0.608795       0.008795          10.5294   

                               ...  diff_wind_speed  real_wind_bearing  \
1970-01-01 00:00:00.000003650  ...         1.234202              358.0   
1970-01-01 00:00:00.000003651  ...         1.453499                0.0   
1970-01-01 00:00:00.000003652  ...         1.196173              350.0   
1970-01-01 00:00:00.000003653  ...         1.312829              333.0   
1970-01-01 00:00:00.000003654  ...         2.071283               80.0   
...                            ...              ...                ...   
1970-01-01 00:00:00.000096447  ...         0.265636               21.0